In [ ]:
# 安裝必要套件
!pip install langgraph openai


In [ ]:
# 導入模組
from langgraph.graph import StateGraph, END
from dataclasses import dataclass
from typing import Dict
from openai import OpenAI
from google.colab import userdata

# 定義狀態結構
@dataclass
class TranslationState:
    input: str
    eng: str = ""
    jpn: str = ""
    kor: str = ""
    feedback: str = ""
    task: str = "translate"
    iteration: int = 0

# 初始化OpenAI客戶端
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))  # 替換為您的API金鑰

# 語言模型調用函數
def llm_call(prompt: str, max_tokens: int = 200) -> str:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens
    )
    return response.choices[0].message.content.strip()

# 翻譯節點：並行生成多語言
def translate_node(state: TranslationState) -> Dict:
    if not state.eng:
        state.eng = llm_call(f"將以下中文翻譯成英文，要求語義精準：\n{state.input}")
    if not state.jpn:
        state.jpn = llm_call(f"將以下中文翻譯成日語，要求自然流暢：\n{state.input}")
    if not state.kor:
        state.kor = llm_call(f"將以下中文翻譯成韓語，要求語法正確：\n{state.input}")
    state.task = "reflect"
    return {"eng": state.eng, "jpn": state.jpn, "kor": state.kor, "task": state.task}

# 反思節點：檢查語義一致性
def reflect_node(state: TranslationState) -> Dict:
    prompt = f"檢查以下翻譯是否語義一致：\n中文：{state.input}\n英文：{state.eng}\n日語：{state.jpn}\n韓語：{state.kor}\n若有問題，回覆具體建議；若無，回覆'無問題'"
    state.feedback = llm_call(prompt)
    state.iteration += 1
    if "無問題" in state.feedback or state.iteration >= 2:
        state.task = "end"
    else:
        state.task = "optimize"
    return {"feedback": state.feedback, "task": state.task, "iteration": state.iteration}

# 優化節點：修正翻譯偏差
def optimize_node(state: TranslationState) -> Dict:
    prompt = f"根據建議修正翻譯：\n建議：{state.feedback}\n原文：{state.input}\n當前：英文-{state.eng}，日語-{state.jpn}，韓語-{state.kor}\n回覆格式：英文;日語;韓語"
    response = llm_call(prompt)
    try:
        eng, jpn, kor = response.split(";")
        state.eng = eng.strip()
        state.jpn = jpn.strip()
        state.kor = kor.strip()
    except ValueError:
        pass  # 避免格式錯誤中斷
    state.task = "reflect"
    return {"eng": state.eng, "jpn": state.jpn, "kor": state.kor, "task": state.task}

# 構建工作流程
workflow = StateGraph(TranslationState)
workflow.add_node("translate", translate_node)
workflow.add_node("reflect", reflect_node)
workflow.add_node("optimize", optimize_node)
workflow.set_entry_point("translate")
workflow.add_conditional_edges(
    "translate",
    lambda state: state.task,
    {"reflect": "reflect", "end": END}
)
workflow.add_conditional_edges(
    "reflect",
    lambda state: state.task,
    {"optimize": "optimize", "end": END}
)
workflow.add_conditional_edges(
    "optimize",
    lambda state: state.task,
    {"reflect": "reflect", "end": END}
)
graph = workflow.compile()

# 執行範例
result = graph.invoke(TranslationState(input="人工智慧改變未來"))
print(f"最終翻譯：\n英文：{result['eng']}\n日語：{result['jpn']}\n韓語：{result['kor']}")
print(f"反饋：{result['feedback']}")